<a href="https://colab.research.google.com/github/ajmoore12/BDB_2025/blob/main/BDB2025_Predictive_Model_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries (if not already installed)
!pip install lightgbm joblib ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.0 MB/s eta 0:00:00


In [5]:
# Import required libraries
import joblib
import numpy as np
import pandas as pd
import lightgbm as lgb
from IPython.display import display
import ipywidgets as widgets
import requests

# URL of the hosted model
model_url = "https://drive.google.com/uc?id=1HZls6AvPREzDjiy_mYMlQwFepiN1yPye"

# Download the model dynamically
response = requests.get(model_url)
if response.status_code == 200:
    with open("model.pkl", "wb") as f:
        f.write(response.content)
    print("Model downloaded successfully!")
else:
    print(f"Failed to download the model. HTTP status code: {response.status_code}")

# Load the model
try:
    model = joblib.load("model.pkl")
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading the model: {e}")

Model downloaded successfully!
Model loaded successfully!


In [7]:
print(model.feature_name_)

['rb_depth', 'TE_shift', 'WR_motion', 'receiverAlignment', 'TE_motion', 'down', 'WR_shift', 'TE_snap_motion', 'WR_snap_motion', 'week', 'qb_depth', 'yardsToGo', 'RB_snap_motion', 'FB_shift', 'FB_motion', 'offenseFormation', 'FB_snap_motion', 'preSnapVisitorTeamWinProbability', 'totalGameSeconds', 'possessionTeam', 'quarter', 'homeTeamPossession', 'RB_motion', 'RB_shift', 'preSnapVisitorScore', 'preSnapHomeScore']


In [15]:
# Convert game clock to totalGameSeconds
def convert_game_clock(clock_str):
    try:
        minutes, seconds = map(int, clock_str.split(':'))
        return minutes * 60 + seconds
    except ValueError:
        raise ValueError("Game Clock must be in 'MM:SS' format.")

# Define a function for user inputs
def get_user_inputs():
    # Widgets for input features
    qb_depth = widgets.FloatSlider(value=5.0, min=0.0, max=15.0, step=0.1, description='QB Depth:')
    rb_depth = widgets.FloatSlider(value=5.0, min=0.0, max=10.0, step=0.1, description='RB Depth:')
    fb_depth = widgets.FloatSlider(value=0.0, min=0.0, max=10.0, step=0.1, description='FB Depth:')
    absolute_yardline = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Abs Yardline:')
    pre_snap_home_wp = widgets.FloatSlider(value=50.0, min=0.0, max=100.0, step=0.1, description='Home WP:')
    pre_snap_visitor_wp = widgets.FloatSlider(value=50.0, min=0.0, max=100.0, step=0.1, description='Visitor WP:')
    offense_formation = widgets.IntSlider(value=3, min=0, max=10, step=1, description='Off Form:')
    receiver_alignment = widgets.IntSlider(value=10, min=0, max=20, step=1, description='Rec Align:')
    pff_pass_coverage = widgets.IntSlider(value=15, min=0, max=20, step=1, description='Pass Coverage:')
    pff_man_zone = widgets.IntSlider(value=1, min=0, max=1, step=1, description='Man Zone:')
    home_team_possession = widgets.ToggleButtons(options=['No', 'Yes'], description='Home Team Poss?')
    quarter = widgets.IntSlider(value=1, min=1, max=5, step=1, description='Quarter:')
    down = widgets.IntSlider(value=1, min=1, max=4, step=1, description='Down:')
    yards_to_go = widgets.IntSlider(value=10, min=0, max=50, step=1, description='Yards To Go:')
    game_clock = widgets.Text(value='15:00', description='Game Clock (MM:SS):')
    pre_snap_home_score = widgets.IntSlider(value=0, min=0, max=50, step=1, description='Home Score:')
    pre_snap_visitor_score = widgets.IntSlider(value=0, min=0, max=50, step=1, description='Visitor Score:')

    # Binary features
    rb_snap_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='RB Snap Motion:')
    wr_snap_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='WR Snap Motion:')
    fb_snap_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='FB Snap Motion:')
    te_snap_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='TE Snap Motion:')
    rb_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='RB Motion:')
    wr_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='WR Motion:')
    fb_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='FB Motion:')
    te_motion = widgets.ToggleButtons(options=['No', 'Yes'], description='TE Motion:')
    week = widgets.IntSlider(value=0, min=0, max=17, step=1, description='Week:')

    # Predict button
    predict_button = widgets.Button(description="Predict")

    # Display input fields
    display(
        qb_depth, rb_depth, fb_depth, absolute_yardline, pre_snap_home_wp, pre_snap_visitor_wp,
        offense_formation, receiver_alignment, pff_pass_coverage, pff_man_zone, home_team_possession,
        quarter, down, yards_to_go, game_clock, pre_snap_home_score, pre_snap_visitor_score,
        rb_snap_motion, wr_snap_motion, fb_snap_motion, te_snap_motion,
        rb_motion, wr_motion, fb_motion, te_motion, week, predict_button
    )

    # Define prediction logic
    def on_predict_clicked(b):
        try:
            # Collect inputs and preprocess
            inputs = [
                qb_depth.value, rb_depth.value, fb_depth.value, absolute_yardline.value,
                pre_snap_home_wp.value, pre_snap_visitor_wp.value, offense_formation.value,
                receiver_alignment.value, pff_pass_coverage.value, pff_man_zone.value,
                1 if home_team_possession.value == 'Yes' else 0, quarter.value, down.value,
                yards_to_go.value, convert_game_clock(game_clock.value),
                pre_snap_home_score.value, pre_snap_visitor_score.value,
                1 if rb_snap_motion.value == 'Yes' else 0, 1 if wr_snap_motion.value == 'Yes' else 0,
                1 if fb_snap_motion.value == 'Yes' else 0, 1 if te_snap_motion.value == 'Yes' else 0,
                1 if rb_motion.value == 'Yes' else 0, 1 if wr_motion.value == 'Yes' else 0,
                1 if fb_motion.value == 'Yes' else 0, 1 if te_motion.value == 'Yes' else 0, week.value
            ]

            # Convert to NumPy array
            inputs = np.array(inputs).reshape(1, -1)

            # Make predictions
            prediction_proba = model.predict_proba(inputs)[0][1]  # Probability for class 1
            prediction = model.predict(inputs)[0]

            print("\nPrediction:")
            print(f"Predicted Class: {prediction}")
            print(f"Probability of Class 1: {prediction_proba:.4f}")
        except Exception as e:
            print(f"Error during prediction: {e}")

    # Link button to prediction logic
    predict_button.on_click(on_predict_clicked)

# Call the function to get user inputs
get_user_inputs()


FloatSlider(value=5.0, description='QB Depth:', max=15.0)

FloatSlider(value=5.0, description='RB Depth:', max=10.0)

FloatSlider(value=0.0, description='FB Depth:', max=10.0)

IntSlider(value=50, description='Abs Yardline:')

FloatSlider(value=50.0, description='Home WP:')

...

Button(description='Predict', style=ButtonStyle())


Prediction:
Predicted Class: 0
Probability of Class 1: 0.1013


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(



Prediction:
Predicted Class: 0
Probability of Class 1: 0.2045


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
